### Se van a normalizar los datos y a reordenar las filas

In [13]:
import pandas as pd
from tqdm.notebook import tqdm
from sklearn import preprocessing
import numpy as np

In [2]:
# Diccionario de prueba
a = {
    'col1': [1,2,3,4,5,6,7,8,9,0],
    'col2': [4,5,6,5,6,7,8,2,3,4]
}

df = pd.DataFrame(a)
df

,col1,col2
0,1,4
1,2,5
2,3,6
3,4,5
4,5,6
5,6,7
6,7,8
7,8,2
8,9,3
9,0,4


In [3]:
columnas = []
profundidades = ['_0','_5','_10']

# añado etiqueta de profundidad
for i in range(3):
    columnas_aux = df.columns + profundidades[i]
    [columnas.append(x) for x in columnas_aux]
    
df_2 = pd.DataFrame(columns=columnas)
df_2

,col1_0,col2_0,col1_5,col2_5,col1_10,col2_10


In [4]:
# concateno datos de la tres profundidades en una sola
for cont in range(int(df.shape[0]/3)):
    inicio = cont*3
    df_aux = df.loc[inicio:inicio+2]
    print(df_aux)
    lista = []

    for fila in df_aux.values.tolist():
        for elem in fila :
            lista.append(elem)
    print(lista)
    df_2.loc[cont] = lista
    

   col1  col2
0     1     4
1     2     5
2     3     6
[1, 4, 2, 5, 3, 6]
   col1  col2
3     4     5
4     5     6
5     6     7
[4, 5, 5, 6, 6, 7]
   col1  col2
6     7     8
7     8     2
8     9     3
[7, 8, 8, 2, 9, 3]


In [5]:
df_2

,col1_0,col2_0,col1_5,col2_5,col1_10,col2_10
0,1,4,2,5,3,6
1,4,5,5,6,6,7
2,7,8,8,2,9,3


# Con estructura avistamientos

In [3]:
# Estructura quitando las columnas de las coordenadas pues no nos sirven
def cargar(nombre):
    df = pd.read_excel(nombre)
    df = df.fillna(method='ffill', axis=0).set_index(['Latitud','Longitud','Fecha','Avistamientos','Profundidad'])#.drop(['level_0','index'],axis = 1)
    # Eliminar columnas Coords
    cols = [c for c in df.columns if 'C' not in c]
    df_inicial=df[cols]

    return df_inicial

In [4]:
profundidades = ['_0','_5','_10']

# añado etiqueta de profundidad
def renombra_atributos(df):
    columnas = []
    for i in range(3):
        columnas_aux = df.columns + profundidades[i]
        [columnas.append(x) for x in columnas_aux]
    df_atributos = pd.DataFrame(columns=columnas)
    return df_atributos
# df_atributos

In [5]:
def unifica_lineas(df):
    df_atributos = renombra_atributos(df)
    # meto tres profundidades consecutivas
    total_lineas = df.shape[0]/3
    pbar = tqdm(total = total_lineas)
    for cont in range(int(df.shape[0]/3)):
        pbar.update(1)
        inicio = cont*3
        df_aux = df.iloc[inicio:inicio+3]
        lista = list()

        for fila in df_aux.values.tolist():
            for elem in fila :
                lista.append(elem)

        dict_atr = dict(zip(df_atributos.columns,lista))
        df_atr_aux = pd.DataFrame(dict_atr,index=[0])
        df_atributos = pd.concat([df_atributos,df_atr_aux])
    pbar.update(0)
    return df_atributos

## normalizar

In [23]:
# sin uso de bibliotecas 
df = pd.read_excel('../4EstructuraFinalSinMissings.xlsx')
df = df.fillna(method='ffill', axis=0).set_index(['Latitud','Longitud','Fecha','Avistamientos','Profundidad'])
df = df['mlotst']
normalized_df=(df-df.min())/(df.max()-df.min())

normalized_df

Latitud     Longitud    Fecha       Avistamientos  Profundidad
-42.666667  -74.083333  2015-05-27  1.0            0.494025       0.296335
                                                   5.078224       0.296335
                                                   9.572997       0.296335
-41.833333  -73.833333  2014-06-04  1.0            0.494025       0.021990
                                                   5.078224       0.021990
                                                                    ...   
-18.500000  -70.333333  2014-08-17  1.0            5.078224       0.028272
                                                   9.572997       0.028272
-18.416667  -70.333333  2015-08-18  1.0            0.494025       0.058639
                                                   5.078224       0.058639
                                                   9.572997       0.058639
Name: mlotst, Length: 1569, dtype: float64

In [24]:
49.592583 / 147.2518157958984

0.336787582088216

In [25]:
n=(1-0)/(2-0)
n=(1--1)/(2--1)
n=(1--1)/(1--1)

n

1.0

In [26]:
# con scikit learn ejemplos


X = [[ 1., -1.,  2.],
     [ 2.,  0.,  0.],
     [ 0.,  1., -1.]]
X_normalized = preprocessing.normalize(X, norm='l2')

X_normalized



array([[ 0.40824829, -0.40824829,  0.81649658],
       [ 1.        ,  0.        ,  0.        ],
       [ 0.        ,  0.70710678, -0.70710678]])

In [27]:
X = [[ 1., -1.,  2.],
     [ 2.,  0.,  0.],
     [ 0.,  1., -1.]]
X_normalized = preprocessing.normalize(X, norm='l1')

X_normalized

array([[ 0.25, -0.25,  0.5 ],
       [ 1.  ,  0.  ,  0.  ],
       [ 0.  ,  0.5 , -0.5 ]])

In [6]:
# con estructura
def normaliza_normalize(df_atributos):
    X = df_atributos.values.tolist()
    x_normalizado = preprocessing.normalize(X, norm='l2')

    x_normalizado

    df_norm = pd.DataFrame(x_normalizado,columns=df_atributos.columns)
    df_norm.to_excel('dfAtributosNormalizado.xlsx')

In [24]:
def normaliza_min_max(df_atributos,nombre):
    X = df_atributos.values.tolist()
    min_max = preprocessing.MinMaxScaler()
    x_normalizado_2 = min_max.fit_transform(X)
    x_normalizado_2
    df_norm = pd.DataFrame(x_normalizado_2,columns=df_atributos.columns)
#     df_norm.to_excel('dfAtributosNormalizado2_{}.xlsx'.format(nombre[3:]))
    return df_norm

## listado avistamientos

In [8]:
def lista_avistamientos(nombre):
    df = pd.read_excel('../4EstructuraFinalSinMissings.xlsx')
    df = df.fillna(method='ffill', axis=0)#.drop(['level_0','index'],axis = 1)
    listado_avistamientos = df.Avistamientos.values.tolist()
    avistamientos = [x for x in listado_avistamientos[::3]]
    df_avistamientos = pd.DataFrame(avistamientos,columns=['Avistamientos'])
    df_avistamientos.to_excel('dfAvistamientos.xlsx')
    return df_avistamientos

In [9]:
# coloca las tres profundidades en una misma linea
def ejecuta(nombre):
    df_inicial = cargar(nombre)
    df_atributos = unifica_lineas(df_inicial)
    df_atri_norm = normaliza_min_max(df_atributos,nombre)
    df_avis_norm = lista_avistamientos(nombre)
    return df_atri_norm,df_avis_norm

In [52]:
df_atri_norm,df_avis_norm = ejecuta('../4EstructuraFinalSinMissings.xlsx')

In [53]:
df_atri_norm.head()

,mlotst_0,zos_0,bottomT_0,thetao_0,so_0,uo_0,vo_0,mlotst_5,zos_5,bottomT_5,...,so_5,uo_5,vo_5,mlotst_10,zos_10,bottomT_10,thetao_10,so_10,uo_10,vo_10
0,0.296335,0.366700,0.462185,0.172787,0.523447,0.478261,0.356818,0.296335,0.366700,0.462185,...,0.460545,0.463016,0.403107,0.296335,0.366700,0.462185,0.175338,0.407069,0.450433,0.418796
1,0.021990,0.339537,0.360498,0.042349,0.534854,1.000000,0.429545,0.021990,0.339537,0.360498,...,0.496126,1.000000,0.460343,0.021990,0.339537,0.360498,0.055174,0.446197,1.000000,0.480627
2,0.056544,0.534205,0.224177,0.151868,0.536882,0.920444,0.248485,0.056544,0.534205,0.224177,...,0.475753,0.905860,0.310711,0.056544,0.534205,0.224177,0.144479,0.427264,0.871030,0.340478
3,0.083770,0.360664,0.465062,0.173490,0.523447,0.484736,0.315909,0.083770,0.360664,0.465062,...,0.463415,0.490874,0.371218,0.083770,0.360664,0.465062,0.178867,0.410855,0.488932,0.389118
4,0.107853,0.338531,0.457759,0.167413,0.525222,0.469010,0.399242,0.107853,0.338531,0.457759,...,0.462554,0.452450,0.439902,0.107853,0.338531,0.457759,0.170270,0.409277,0.437921,0.451772


In [54]:
df_avis_norm.head()

,Avistamientos
0,1.0
1,1.0
2,1.0
3,20.0
4,151.0


In [15]:
for i in [0,7,14,30,45,60]:
    df_atri_norm,df_avis_norm = ejecuta('../-EstructuraFinal{}dias.xlsx'.format(i))

In [26]:
df = pd.read_excel('../3estruct_0dias_0celdas.xlsx').set_index(['Latitud','Longitud','Fecha','Avistamientos','Profundidad'])
df = unifica_lineas(df)
# df = df['mlotst_0']
df = normaliza_min_max(df,'')
df

ValueError: could not convert string to float: '[-43.33333333333333, -74.16666666666666]'